In [1]:
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import random
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neural_network import MLPClassifier

### IDEA
La idea de este notebook es probar solo con los datos relacionados al area.

In [2]:
# En base a lo visto por el TP1 las siguientes areas son las mas
# postuladas
#areas_importantes = [
#    'Ventas', 'Administración', 'Producción', 'Comercial', 'Atención al Cliente',
#    'Recepcionista', 'Call Center', 'Telemarketing', 'Tesorería', 'Mantenimient y Limpieza',
#]

# 2.  AVISOS

In [3]:
avisos = pd.concat([
    pd.read_csv("../input/datos/fiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/MISfiuba_6_avisos_detalle_missing_nivel_laboral.csv")
])
avisos.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [4]:
# Solo me quedo con nombre de area
avisos = avisos[['idaviso', 'nombre_area', 'nombre_zona', 'tipo_de_trabajo']]
avisos.head()

,idaviso,nombre_area,nombre_zona,tipo_de_trabajo
0,8725750,Comercial,Gran Buenos Aires,Full-time
1,17903700,Salud,Gran Buenos Aires,Full-time
2,1000150677,Transporte,Capital Federal,Full-time
3,1000610287,Transporte,Gran Buenos Aires,Full-time
4,1000872556,Producción,Gran Buenos Aires,Full-time


In [5]:
# Cargamos previamente los datos de postulaciones para encontrar las areas mas postuladas
postulaciones = pd.concat([
    pd.read_csv('../input/datos/fiuba_4_postulaciones.csv'),
    pd.read_csv('../input/datosdatos/ENTfiuba_4_postulaciones.csv'),
])
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [6]:
mas_postulados = pd.merge(postulaciones[['idaviso']], avisos[['idaviso', 'nombre_area']], on='idaviso', how='inner')
mas_postulados.head()

,idaviso,nombre_area
0,1112257047,Atención al Cliente
1,1112257047,Atención al Cliente
2,1112257047,Atención al Cliente
3,1112257047,Atención al Cliente
4,1112257047,Atención al Cliente


In [7]:
areas_importantes = mas_postulados['nombre_area'].value_counts().head(20).index

In [8]:
avisos.drop_duplicates(subset='idaviso', inplace=True)
avisos['nombre_area'] = avisos['nombre_area'].apply(lambda x: x if x in areas_importantes else 'Otra Area')

In [9]:
# Uso one hot encoding para area
dummies_area = pd.get_dummies(avisos['nombre_area'])
avisos = pd.concat([avisos, dummies_area], axis=1)
avisos.drop(columns='nombre_area', inplace=True)
avisos.head()

,idaviso,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas
0,8725750,Gran Buenos Aires,Full-time,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,17903700,Gran Buenos Aires,Full-time,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,1000150677,Capital Federal,Full-time,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,1000610287,Gran Buenos Aires,Full-time,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,1000872556,Gran Buenos Aires,Full-time,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [10]:
# ZONA
# 1 GBA
# 0 Capital y otros
avisos['nombre_zona'] = avisos['nombre_zona'].apply(lambda x: 1 if x == 'Gran Buenos Aires' else 0)

In [11]:
# Tipo de trabajo
# 1 Full-time
# 0 otros
avisos['tipo_de_trabajo'] = avisos['tipo_de_trabajo'].apply(lambda x: 1 if x == 'Full-time' else 0)

In [12]:
avisos.head()

,idaviso,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas
0,8725750,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,17903700,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,1000150677,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,1000610287,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,1000872556,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


# 2. VISTAS

In [13]:
vistas = pd.concat([
    pd.read_csv("../input/datos/fiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_3_vistas.csv")
])
vistas.head()

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [14]:
vistas.rename(columns={'idAviso':'idaviso'}, inplace=True)
vistas = vistas[['idaviso', 'idpostulante']]
vistas.drop_duplicates(subset=['idaviso', 'idpostulante'], inplace=True)
vistas.head()

,idaviso,idpostulante
0,1111780242,YjVJQ6Z
1,1112263876,BmVpYoR
2,1112327963,wVkBzZd
3,1112318643,OqmP9pv
4,1111903673,DrpbXDP


In [16]:
# Aniado la informacion del area de cada aviso
avisos_vistos = pd.merge(vistas, 
                         avisos.drop(columns=['nombre_zona','tipo_de_trabajo']),
                         on='idaviso', 
                         how='inner')
avisos_vistos.head()

,idaviso,idpostulante,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas
0,1111780242,YjVJQ6Z,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,1111780242,dYjRpMD,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,1111780242,wVopPXl,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,1111780242,wVoOPoL,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,1111780242,KBdJJKd,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [17]:
avisos_vistos.drop(columns='idaviso',inplace=True)

In [18]:
# Cada columna representa la cantidad de avisos vistos por area para cada postulante
avisos_vistos = avisos_vistos.pivot_table(index='idpostulante', aggfunc='sum')
avisos_vistos.head()

,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas
idpostulante,,,,,,,,,,,,,,,,,,,,,
0005E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00Lkv,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00dMd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01QAq,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
021OM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Ningun usuario supera 2^16 cantidad de vistas por area
avisos_vistos = avisos_vistos.astype('uint16')

In [20]:
# le agrego la palabra 'vistas' adelante para que sea mas claro y no se confunda luego con el area postulada
nuevos_nombres = []
for area in avisos_vistos.columns:
    area = "vistas " + area
    nuevos_nombres.append(area)
avisos_vistos.columns = nuevos_nombres
avisos_vistos.head()

,vistas Administración,vistas Administración de Personal,vistas Almacén / Depósito / Expedición,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Contabilidad,vistas Créditos y Cobranzas,vistas Gastronomia,vistas Legal,vistas Logística,vistas Mantenimiento y Limpieza,vistas Marketing,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Recursos Humanos,vistas Secretaria,vistas Telemarketing,vistas Tesorería,vistas Ventas
idpostulante,,,,,,,,,,,,,,,,,,,,,
0005E,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
00Lkv,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
00dMd,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
01QAq,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
021OM,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


# 3. GENERO Y EDAD

In [21]:
genyedad = pd.concat([
    pd.read_csv("../input/datos/fiuba_2_postulantes_genero_y_edad.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_2_postulantes_genero_y_edad.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_2_postulantes_genero_y_edad.csv")
])
genyedad.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [22]:
# elimino los datos repetidos
genyedad.drop_duplicates(subset='idpostulante', inplace=True)
genyedad.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [23]:
# 1 MASC
# 0 FEM + no declara
genyedad['sexo'] = genyedad['sexo'].apply(lambda x: 1 if x == 'MASC' else 0)

In [24]:
# Calculo las edades
genyedad['fecha_dt'] = pd.to_datetime(genyedad['fechanacimiento'], errors='coerce')
genyedad.head()

,idpostulante,fechanacimiento,sexo,fecha_dt
0,NM5M,1970-12-03,0,1970-12-03
1,5awk,1962-12-04,0,1962-12-04
2,ZaO5,1978-08-10,0,1978-08-10
3,NdJl,1969-05-09,1,1969-05-09
4,eo2p,1981-02-16,1,1981-02-16


In [25]:
genyedad['edad'] = 2018 - genyedad['fecha_dt'].dt.year
genyedad.head()

,idpostulante,fechanacimiento,sexo,fecha_dt,edad
0,NM5M,1970-12-03,0,1970-12-03,48.0
1,5awk,1962-12-04,0,1962-12-04,56.0
2,ZaO5,1978-08-10,0,1978-08-10,40.0
3,NdJl,1969-05-09,1,1969-05-09,49.0
4,eo2p,1981-02-16,1,1981-02-16,37.0


In [26]:
genyedad.drop(columns=['fechanacimiento', 'fecha_dt'], inplace=True)
genyedad.head()

,idpostulante,sexo,edad
0,NM5M,0,48.0
1,5awk,0,56.0
2,ZaO5,0,40.0
3,NdJl,1,49.0
4,eo2p,1,37.0


In [27]:
genyedad.fillna(genyedad['edad'].mean(), inplace=True)
genyedad = genyedad.astype({'sexo':'int8', 'edad':'uint8'})
genyedad.head()

,idpostulante,sexo,edad
0,NM5M,0,48
1,5awk,0,56
2,ZaO5,0,40
3,NdJl,1,49
4,eo2p,1,37


In [28]:
# lo uno con avisos_vistos
usuarios = pd.merge(avisos_vistos, genyedad, on='idpostulante', how='inner')
usuarios.head()

,idpostulante,vistas Administración,vistas Administración de Personal,vistas Almacén / Depósito / Expedición,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Contabilidad,vistas Créditos y Cobranzas,vistas Gastronomia,vistas Legal,vistas Logística,vistas Mantenimiento y Limpieza,vistas Marketing,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Recursos Humanos,vistas Secretaria,vistas Telemarketing,vistas Tesorería,vistas Ventas,sexo,edad
0,0z5Dmrd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,53
1,0z5JW1r,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47
2,0z5VvGv,1,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,31
3,0zB01pE,0,0,0,0,0,1,1,0,0,0,0,0,0,3,0,0,0,0,0,0,0,1,29
4,0zB026d,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,40


In [29]:
# Con estos daatos caracterizamos a los usuarios

# 3. POSTULACIONES

In [30]:
# Cargamos los datos antes para obtener las areas mas postuladas
postulaciones = pd.concat([
    pd.read_csv('../input/datos/fiuba_4_postulaciones.csv'),
    pd.read_csv('../input/datosdatos/ENTfiuba_4_postulaciones.csv'),
])
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [31]:

# La carga de datos se hizo anteriormente
postulaciones.drop(columns='fechapostulacion', inplace=True)
postulaciones.head()

,idaviso,idpostulante
0,1112257047,NM5M
1,1111920714,NM5M
2,1112346945,NM5M
3,1112345547,NM5M
4,1112237522,5awk


In [32]:
postulaciones.drop_duplicates(subset=['idaviso', 'idpostulante'], inplace=True)

In [33]:
postulaciones.head()

,idaviso,idpostulante
0,1112257047,NM5M
1,1111920714,NM5M
2,1112346945,NM5M
3,1112345547,NM5M
4,1112237522,5awk


In [34]:
# Busco los avisos y las areas de cada postulacion
postulaciones_detalle = pd.merge(postulaciones, avisos, on='idaviso', how='inner')

In [35]:
# 1 en el area de la categoria
postulaciones_detalle.head()

,idaviso,idpostulante,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas
0,1112257047,NM5M,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1112257047,1kJqGb,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1112257047,eOE9Rr,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1112257047,Zrx8Xz,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1112257047,ZrKNQY,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
# Finalmente uno la informacion de los usuarios y sus postulaciones
datos = pd.merge(usuarios, postulaciones_detalle, on='idpostulante', how='inner')
datos.head()

,idpostulante,vistas Administración,vistas Administración de Personal,vistas Almacén / Depósito / Expedición,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Contabilidad,vistas Créditos y Cobranzas,vistas Gastronomia,vistas Legal,vistas Logística,vistas Mantenimiento y Limpieza,vistas Marketing,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Recursos Humanos,vistas Secretaria,vistas Telemarketing,vistas Tesorería,vistas Ventas,sexo,edad,idaviso,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas
0,0z5Dmrd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,53,1112384041,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0z5Dmrd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,53,1112420060,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0z5JW1r,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1112292169,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0z5JW1r,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1112309728,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0z5JW1r,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1112257171,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
# Todos estos avisos fueron postulaciones y por lo tanto se clasifican como postulados
# POSTULADO 1
# NO_POSTULADO 0
datos['postulado'] = 1
datos.head()

,idpostulante,vistas Administración,vistas Administración de Personal,vistas Almacén / Depósito / Expedición,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Contabilidad,vistas Créditos y Cobranzas,vistas Gastronomia,vistas Legal,vistas Logística,vistas Mantenimiento y Limpieza,vistas Marketing,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Recursos Humanos,vistas Secretaria,vistas Telemarketing,vistas Tesorería,vistas Ventas,sexo,edad,idaviso,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas,postulado
0,0z5Dmrd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,53,1112384041,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,0z5Dmrd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,53,1112420060,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0z5JW1r,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1112292169,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,0z5JW1r,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1112309728,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0z5JW1r,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1112257171,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [38]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5117287 entries, 0 to 5117286
Data columns (total 49 columns):
idpostulante                              object
vistas Administración                     uint16
vistas Administración de Personal         uint16
vistas Almacén / Depósito / Expedición    uint16
vistas Atención al Cliente                uint16
vistas Call Center                        uint16
vistas Comercial                          uint16
vistas Contabilidad                       uint16
vistas Créditos y Cobranzas               uint16
vistas Gastronomia                        uint16
vistas Legal                              uint16
vistas Logística                          uint16
vistas Mantenimiento y Limpieza           uint16
vistas Marketing                          uint16
vistas Otra Area                          uint16
vistas Producción                         uint16
vistas Recepcionista                      uint16
vistas Recursos Humanos                   uint16
vista

# 4. Generacion de NO_POSTULACIONES

In [39]:
# En realidad es la info que describe a los usuarios segun sus vistas.
# TODO cambiar el nombre de avisos_vistos a usuarios
usuarios.head()

,idpostulante,vistas Administración,vistas Administración de Personal,vistas Almacén / Depósito / Expedición,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Contabilidad,vistas Créditos y Cobranzas,vistas Gastronomia,vistas Legal,vistas Logística,vistas Mantenimiento y Limpieza,vistas Marketing,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Recursos Humanos,vistas Secretaria,vistas Telemarketing,vistas Tesorería,vistas Ventas,sexo,edad
0,0z5Dmrd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,53
1,0z5JW1r,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47
2,0z5VvGv,1,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,31
3,0zB01pE,0,0,0,0,0,1,1,0,0,0,0,0,0,3,0,0,0,0,0,0,0,1,29
4,0zB026d,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,40


In [40]:
# Categoria de cada aviso
avisos.head()

,idaviso,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas
0,8725750,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,17903700,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,1000150677,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,1000610287,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,1000872556,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [41]:
# Mezclo al azar usuarios con sus vistas mas algun aviso cualquiera.
N = 5000000
usuarios_random = usuarios.sample(n=N, replace=True, random_state=44)
avisos_random = avisos.sample(n=N, replace=True, random_state=99)

In [42]:
usuarios_random.reset_index(inplace=True)
avisos_random.reset_index(inplace=True)
usuarios_random.drop(columns='index', inplace=True)
avisos_random.drop(columns='index', inplace=True)

In [43]:
avisos_random.size

120000000

In [44]:
# No se porque tienen tamanios distintos asi que solo tomo los primeros N de 
# avisos_random
datos_random = pd.concat([usuarios_random, avisos_random], axis=1)

In [45]:
datos_random.head()

,idpostulante,vistas Administración,vistas Administración de Personal,vistas Almacén / Depósito / Expedición,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Contabilidad,vistas Créditos y Cobranzas,vistas Gastronomia,vistas Legal,vistas Logística,vistas Mantenimiento y Limpieza,vistas Marketing,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Recursos Humanos,vistas Secretaria,vistas Telemarketing,vistas Tesorería,vistas Ventas,sexo,edad,idaviso,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas
0,1QP3WaE,0,0,0,0,0,0,0,1,0,0,0,0,0,10,0,0,0,0,0,0,0,0,36,1112397732,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,QNaOVbl,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,1112342096,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,a52DEq,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,38,1112225199,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,LNK6OA5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5,0,31,1112322633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,5mPdQMX,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,31,1112287852,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [46]:
# POSTULADO 1
# NO_POSTULADO 0
datos_random['postulado'] = 0
datos_random.head()

,idpostulante,vistas Administración,vistas Administración de Personal,vistas Almacén / Depósito / Expedición,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Contabilidad,vistas Créditos y Cobranzas,vistas Gastronomia,vistas Legal,vistas Logística,vistas Mantenimiento y Limpieza,vistas Marketing,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Recursos Humanos,vistas Secretaria,vistas Telemarketing,vistas Tesorería,vistas Ventas,sexo,edad,idaviso,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas,postulado
0,1QP3WaE,0,0,0,0,0,0,0,1,0,0,0,0,0,10,0,0,0,0,0,0,0,0,36,1112397732,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,QNaOVbl,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,1112342096,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,a52DEq,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,38,1112225199,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,LNK6OA5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5,0,31,1112322633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,5mPdQMX,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,31,1112287852,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [47]:
# Estos ultimos son los datos no postulados
datos_random.drop_duplicates(subset=['idpostulante', 'idaviso'],inplace=True)
datos_random.head()

,idpostulante,vistas Administración,vistas Administración de Personal,vistas Almacén / Depósito / Expedición,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Contabilidad,vistas Créditos y Cobranzas,vistas Gastronomia,vistas Legal,vistas Logística,vistas Mantenimiento y Limpieza,vistas Marketing,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Recursos Humanos,vistas Secretaria,vistas Telemarketing,vistas Tesorería,vistas Ventas,sexo,edad,idaviso,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas,postulado
0,1QP3WaE,0,0,0,0,0,0,0,1,0,0,0,0,0,10,0,0,0,0,0,0,0,0,36,1112397732,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,QNaOVbl,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,1112342096,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,a52DEq,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,38,1112225199,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,LNK6OA5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5,0,31,1112322633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,5mPdQMX,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,31,1112287852,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


# DATOS FINALES

In [48]:
final100k = pd.read_csv("../input/finalfinal/test_final_100k.csv")
# Recolecto su informacion en los distintos dataFrames

# Agrego datos de usuario
final = pd.merge(final100k, usuarios, on='idpostulante', how='left')
final.fillna(0, inplace=True)

# agrego datos de avisos
final = pd.merge(final, avisos, on='idaviso', how='left')
final.fillna(0, inplace=True)
X_final = final.drop(columns=['id', 'idaviso', 'idpostulante']) 
X_final.head()

,vistas Administración,vistas Administración de Personal,vistas Almacén / Depósito / Expedición,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Contabilidad,vistas Créditos y Cobranzas,vistas Gastronomia,vistas Legal,vistas Logística,vistas Mantenimiento y Limpieza,vistas Marketing,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Recursos Humanos,vistas Secretaria,vistas Telemarketing,vistas Tesorería,vistas Ventas,sexo,edad,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas
0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,42.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,31.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,36.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,32.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


# 5. Datos de entrenamiento para el algoritmo

In [49]:
# Me creo un DF con los datos postulados y no postulados
df = pd.concat([datos,datos_random])
df.drop_duplicates(subset=['idpostulante', 'idaviso'], inplace=True)
df.head()

,idpostulante,vistas Administración,vistas Administración de Personal,vistas Almacén / Depósito / Expedición,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Contabilidad,vistas Créditos y Cobranzas,vistas Gastronomia,vistas Legal,vistas Logística,vistas Mantenimiento y Limpieza,vistas Marketing,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Recursos Humanos,vistas Secretaria,vistas Telemarketing,vistas Tesorería,vistas Ventas,sexo,edad,idaviso,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas,postulado
0,0z5Dmrd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,53,1112384041,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,0z5Dmrd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,53,1112420060,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0z5JW1r,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1112292169,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,0z5JW1r,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1112309728,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0z5JW1r,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1112257171,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [50]:
# X Datos
# y Categorias
X = df.drop(columns=['idpostulante', 'idaviso', 'postulado'])
# adicionales
X.head()

,vistas Administración,vistas Administración de Personal,vistas Almacén / Depósito / Expedición,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Contabilidad,vistas Créditos y Cobranzas,vistas Gastronomia,vistas Legal,vistas Logística,vistas Mantenimiento y Limpieza,vistas Marketing,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Recursos Humanos,vistas Secretaria,vistas Telemarketing,vistas Tesorería,vistas Ventas,sexo,edad,nombre_zona,tipo_de_trabajo,Administración,Administración de Personal,Almacén / Depósito / Expedición,Atención al Cliente,Call Center,Comercial,Contabilidad,Créditos y Cobranzas,Gastronomia,Legal,Logística,Mantenimiento y Limpieza,Marketing,Otra Area,Producción,Recepcionista,Recursos Humanos,Secretaria,Telemarketing,Tesorería,Ventas
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,53,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,53,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,0,0,0,2,1,47,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
y = df['postulado']

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [53]:
scaler = StandardScaler()  
scaler.fit(X_train)
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

In [ ]:
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
clf = Perceptron(n_iter=40, eta0=0.1, random_state=0)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)  
print('accuracy: {0:.2f}%'.format(accuracy_score(y_test,y_pred)*100))

# 7. PREDICCION

In [ ]:
y_final = clf.predict(X_final)

In [ ]:
y_final

# 8. SUMMIT

In [ ]:
y_summit = pd.DataFrame(y_final)
y_summit.columns = ['sepostulo']
y_summit.head()

In [ ]:
y_summit.info()

In [ ]:
summit = y_summit.reset_index()
summit.head()

In [ ]:
summit.rename(columns={'index':'id'}, inplace=True)
summit.head()

In [ ]:
# Guardo los datos
summit.to_csv("summitGSNB.csv", index=False)
